In [1]:
import pandas as pd
import numpy as np
import os
import timeit
from tqdm.notebook import trange, tqdm

In [2]:
CHARTEVENTS_BY_ICUSTAY_ID = 'data/parquet/'
REDUCED_CE_BY_ICUSTAY_ID = 'data/samples/'
ICUSTAY_ID_TEXT = 'ICUSTAY_ID'
MAX_HOURS = 48

In [3]:
chartevents_dir_list = os.listdir(CHARTEVENTS_BY_ICUSTAY_ID)
len(chartevents_dir_list)

48251

In [4]:
event_to_id =[
{'CHAREVENT':'GCS_EYE', 'DESCRIPTION':'','ITEMID':[184, 220739],'UNIT':['NONE','NONE']},
{'CHAREVENT':'GCS_MOTOR', 'DESCRIPTION':'','ITEMID':[454, 223901],'UNIT':['NONE','NONE']},
{'CHAREVENT':'GCS_VERBAL', 'DESCRIPTION':'','ITEMID':[723, 223900],'UNIT':['NONE','NONE']},
{'CHAREVENT':'GCS_TOTAL', 'DESCRIPTION':'Sum of the 3 GCS events','ITEMID':[198],'UNIT':['NONE']},
{'CHAREVENT':'CAPILLARY_REFILL', 'DESCRIPTION':'','ITEMID':[3348, 115, 8377, 223951, 224308],'UNIT':['BINARY','BINARY','BINARY','BINARY','BINARY']},
{'CHAREVENT':'D_BLOOD_PRESSURE', 'DESCRIPTION':'','ITEMID':[8368, 220051, 225310, 8555, 8441, 220180, 8502, 8440, 8503, 8504, 8507, 8506, 224643, 227242],'UNIT':['mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg']},
{'CHAREVENT':'M_BLOOD_PRESSURE', 'DESCRIPTION':'','ITEMID':[52, 220052, 225312, 224, 6702, 224322, 456, 220181, 3312, 3314, 3316, 3322, 3320],'UNIT':['mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg']},
{'CHAREVENT':'S_BLOOD_PRESSURE', 'DESCRIPTION':'','ITEMID':[51, 220050, 225309, 6701, 455, 220179, 3313, 3315, 442, 3317, 3323, 3321, 224167, 227243],'UNIT':['mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg','mmHg']},
{'CHAREVENT':'HEART_RATE', 'DESCRIPTION':'','ITEMID':[211, 220045],'UNIT':['bpm','bpm']},
{'CHAREVENT':'GLUCOSE', 'DESCRIPTION':'','ITEMID':[807, 811, 1529, 3745, 225664, 220621, 226537, 3447, 3816, 3744, 227015, 227016, 1310, 1455, 2338, 1812, 228388],'UNIT':['','']},
{'CHAREVENT':'FRAC_OXYGEN', 'DESCRIPTION':'','ITEMID':[3420, 223835, 3422, 189, 727],'UNIT':['%','%','%','%','%']},
{'CHAREVENT':'O2_SAT', 'DESCRIPTION':'','ITEMID':[834, 8498, 220227, 646, 220277],'UNIT':['%','%','%','%','%']},
{'CHAREVENT':'RESP_RATE', 'DESCRIPTION':'','ITEMID':[618, 220210, 3603, 224689, 614, 651, 224422, 615, 224690, 619, 224688],'UNIT':['insp/min','']},
{'CHAREVENT':'BODY_TEMP', 'DESCRIPTION':'','ITEMID':[3655, 677, 676, 223762, 3654, 678, 223761, 679, 8537, 645, 591, 226329, 597, 227054, 228242],'UNIT':['C','C','C','C','F','F','F','F','C','','','','C','','F']},
{'CHAREVENT':'PH', 'DESCRIPTION':'','ITEMID':[3839, 1673, 780, 1126, 223830, 4753, 4202, 860, 220274, 8387, 1880, 3777, 227586, 8385, 1352, 4755, 7966],'UNIT':['NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE','NONE']},
{'CHAREVENT':'WEIGHT', 'DESCRIPTION':'','ITEMID':[763, 224639, 226512, 3580, 3693, 3581, 226531, 3582],'UNIT':['?','kg','kg','kg','kg','lb','lb','oz']},
{'CHAREVENT':'HEIGHT', 'DESCRIPTION':'','ITEMID':[226707, 226730, 1394],'UNIT':['?','cm','in']},
]

In [5]:
itemid_list =[]
for d in event_to_id:
    a = np.array(d['ITEMID']).astype(np.int64).tolist()
    print(a)


[184, 220739]
[454, 223901]
[723, 223900]
[198]
[3348, 115, 8377, 223951, 224308]
[8368, 220051, 225310, 8555, 8441, 220180, 8502, 8440, 8503, 8504, 8507, 8506, 224643, 227242]
[52, 220052, 225312, 224, 6702, 224322, 456, 220181, 3312, 3314, 3316, 3322, 3320]
[51, 220050, 225309, 6701, 455, 220179, 3313, 3315, 442, 3317, 3323, 3321, 224167, 227243]
[211, 220045]
[807, 811, 1529, 3745, 225664, 220621, 226537, 3447, 3816, 3744, 227015, 227016, 1310, 1455, 2338, 1812, 228388]
[3420, 223835, 3422, 189, 727]
[834, 8498, 220227, 646, 220277]
[618, 220210, 3603, 224689, 614, 651, 224422, 615, 224690, 619, 224688]
[3655, 677, 676, 223762, 3654, 678, 223761, 679, 8537, 645, 591, 226329, 597, 227054, 228242]
[3839, 1673, 780, 1126, 223830, 4753, 4202, 860, 220274, 8387, 1880, 3777, 227586, 8385, 1352, 4755, 7966]
[763, 224639, 226512, 3580, 3693, 3581, 226531, 3582]
[226707, 226730, 1394]


In [6]:
items =[]
for d in event_to_id:
    items.extend(d['ITEMID'])
items = np.array(items).astype(np.int64).tolist()

In [7]:
def size_list(l: list) -> list:
    out = l.copy()
    length = len(out)
    if length > MAX_HOURS:
        out = out[:48]
    elif length < MAX_HOURS:
        out[:0] = [out[0]]*(MAX_HOURS-length)
    return out

In [8]:
# %%timeit

# test = pd.read_parquet(CHARTEVENTS_BY_ICUSTAY_ID + chartevents_dir_list[0])
# test = test.reset_index(drop=True)
# # TODO:need to insert try block, capture failed cases by ICUSTAY_ID
# test['ITEMID'] = test['ITEMID'].astype(np.int64)


In [9]:
test = pd.read_parquet(CHARTEVENTS_BY_ICUSTAY_ID + chartevents_dir_list[0])
test = test.reset_index(drop=True)
# TODO:need to insert try block, capture failed cases by ICUSTAY_ID
test['ITEMID'] = test['ITEMID'].astype(np.int64)

In [10]:
test.head(20)

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,55973,152234,220277,2181-11-28 12:00:00,2181-11-28 12:42:00,17474,94,94,%,0,0,None,None
1,55973,152234,223761,2181-11-28 12:00:00,2181-11-28 12:43:00,17474,96.4,96.4,?F,0,0,None,None
2,55973,152234,220045,2181-11-28 12:10:00,2181-11-28 14:53:00,17474,66,66,bpm,0,0,None,None
3,55973,152234,220050,2181-11-28 12:10:00,2181-11-28 14:53:00,17474,93,93,mmHg,0,0,None,None
4,55973,152234,220051,2181-11-28 12:10:00,2181-11-28 14:53:00,17474,43,43,mmHg,0,0,None,None
5,55973,152234,220052,2181-11-28 12:10:00,2181-11-28 14:53:00,17474,59,59,mmHg,0,0,None,None
6,55973,152234,220210,2181-11-28 12:10:00,2181-11-28 14:53:00,17474,14,14,insp/min,0,0,None,None
7,55973,152234,220277,2181-11-28 12:10:00,2181-11-28 14:53:00,17474,99,99,%,0,0,None,None
8,55973,152234,220045,2181-11-28 12:20:00,2181-11-28 14:53:00,17474,65,65,bpm,0,0,None,None
9,55973,152234,220050,2181-11-28 12:20:00,2181-11-28 14:53:00,17474,95,95,mmHg,0,0,None,None


In [11]:
test.dtypes

SUBJECT_ID               int64
HADM_ID                  int64
ITEMID                   int64
CHARTTIME       datetime64[ns]
STORETIME               object
CGID                    object
VALUE                   object
VALUENUM                object
VALUEUOM                object
WARNING                 object
ERROR                   object
RESULTSTATUS            object
STOPPED                 object
dtype: object

In [12]:
test.shape

(2863, 13)

In [13]:
test['HADM_ID'].nunique()

1

In [14]:
test = test[test['ITEMID'].isin(items)]
test.shape

(706, 13)

In [15]:
test = test.sort_values(['CHARTTIME'])

In [16]:
test['CHARTTIME'].max()

Timestamp('2181-11-28 20:04:00')

In [17]:
(test['CHARTTIME'].max() - test['CHARTTIME'].min()) //  np.timedelta64(1, "h")

72

In [18]:
# Index 0 is oldest hour record
hour_df_list = [g.reset_index(drop=True) for i, g in test.groupby([(test['CHARTTIME'].max() - test['CHARTTIME']) //  np.timedelta64(1, "h")])]

In [19]:
len(hour_df_list)

71

In [20]:
hour_df_list = size_list(hour_df_list)
len(hour_df_list)

48

In [21]:
sample_columns = []
sample_columns_ind = []
for d in event_to_id:
    sample_columns.append(d['CHAREVENT'])
    sample_columns.append(d['CHAREVENT']+'_ID')
    sample_columns_ind.append(d['CHAREVENT']+'_IND')

sample_columns.extend(sample_columns_ind)
len(sample_columns)

51

In [22]:
nan_row = list(2 * len(event_to_id) * [None])
indicator_row = list(len(event_to_id) * [0])
print(len(nan_row))
print(len(indicator_row))

34
17


In [23]:
chartevents_dir_list = os.listdir(CHARTEVENTS_BY_ICUSTAY_ID)

In [24]:
# cycle over all ICUSTAY_IDs in directory, get events and sort by hour,
# keep last 48 hours, or duplicate last hour until 48 entries
failed = []
for stay in tqdm(chartevents_dir_list, total=len(chartevents_dir_list)):
    try:
        test = pd.read_parquet(CHARTEVENTS_BY_ICUSTAY_ID + stay)
        test = test.reset_index(drop=True)
        # TODO:need to insert try block, capture failed cases by ICUSTAY_ID
        test['ITEMID'] = test['ITEMID'].astype(np.int64)
        # remove non-needed ITEMIDs
        test = test[test['ITEMID'].isin(items)]
        test = test.sort_values(['CHARTTIME'])
        hour_df_list = [g.reset_index(drop=True) for i, g in test.groupby([(test['CHARTTIME'].max() - test['CHARTTIME']) //  np.timedelta64(1, "h")])]
        hour_df_list = size_list(hour_df_list)
        hour_df_list = hour_df_list[::-1]

        # Cycle over the list of hour dfs, store data into 1 df

        # event_to_id

        sample_df = pd.DataFrame(columns=sample_columns)
        for df in hour_df_list:
            new_row = nan_row.copy()
            new_ind = indicator_row.copy()
            for i, event in enumerate(event_to_id):
                temp = df[df['ITEMID'].isin(event['ITEMID'])]
                
                if temp.shape[0] > 0:
                    
                    new_row[i*2] = temp.iloc[0]['VALUE']
                    new_row[i*2 + 1] = temp.iloc[0]['ITEMID']
                    new_ind[i] = 1
                    # print(new_row)

            new_row.extend(new_ind)
            sample_df.loc[len(sample_df.index)] = new_row


        sample_df.to_parquet(REDUCED_CE_BY_ICUSTAY_ID+stay)
    except Exception as e:
        failed.append(stay)
        failed.append(str(e))
        continue




  0%|          | 0/48251 [00:00<?, ?it/s]

In [27]:
len(failed)

352

In [29]:
failed[1]

'list index out of range'

In [25]:
# sample_df = pd.DataFrame(columns=sample_columns)
# sample_df